# Let's roll our own historical partisan lean

### historical partisan lean by congressional district is surprisingly difficult to come by. 
Returns of presidential elections are tabulated differently from congressional elections, meaning partisan lean data is difficult to calculate and proprietary. Many data-sets at the district level are not freely available (the [OpenElections](http://www.openelections.net/about/) project is attempting to compile an open dataset). This notebook should create a rough approximation.

### Import Data:

In [1]:
!ls Data/

102slersuoacontest20181024-1.tab
102slersuoacontest20181024codebook-1.txt
1976-2018-house.csv
averages.csv
Cache
county_assignments.csv
countypres_2000-2016.csv
fivethirtyeight_partisan_lean_DISTRICTS.csv
fivethirtyeight_partisan_lean_STATES.csv
Output
vote_predictions.csv


In [2]:
#Use election returns available from https://electionlab.mit.edu/data
#combine together with county lists available from https://github.com/fivethirtyeight/redistricting-atlas-data
import pandas as pd

house_by_district = pd.read_csv('Data/1976-2018-house.csv', encoding = 'unicode-escape')
house_by_district.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,Bill Davenport,democrat,False,total,58906,157170,False,20171005
1,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,Jack Edwards,republican,False,total,98257,157170,False,20171005
2,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,NaN,NaN,True,total,7,157170,False,20171005
3,1976,Alabama,AL,1,63,41,US House,2,gen,False,False,J. Carole Keahey,democrat,False,total,66288,156362,False,20171005
4,1976,Alabama,AL,1,63,41,US House,2,gen,False,False,NaN,NaN,True,total,5,156362,False,20171005


In [3]:
pres_by_county = pd.read_csv('Data/countypres_2000-2016.csv')
pres_by_county.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20190722


In [4]:
county_district = pd.read_csv('Data/county_assignments.csv')
#Narrow dataset down to current instead of theoretical districts 
current = county_district.maptype == 'current'
county_district = county_district.loc[current,:]

#remove the word "county" from county names
county_district.county = county_district.county.map(lambda x: x.split('County')[0].strip())

#create a column that matchs other FIPS codes
county_district['FIPS'] = county_district.countyfp + county_district.statefp * 1000

#display
county_district.head()

,statefp,state,maptype,countyfp,county,cd,FIPS
24440,1,AL,current,1,Autauga,2,1001
24441,1,AL,current,3,Baldwin,1,1003
24442,1,AL,current,5,Barbour,2,1005
24443,1,AL,current,7,Bibb,6,1007
24444,1,AL,current,9,Blount,6,1009


In [5]:
#State Legislative Election Returns, 1967-2016: Restructured For Use
#https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DRSACA

state_leg= pd.read_csv('Data/102slersuoacontest20181024-1.tab', sep='\t').loc[:,['year','state','sfips','dno','sen','sab','dvote','rvote','ovote']]

state_leg.head()

/home/nhorning/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,state,sfips,dno,sen,sab,dvote,rvote,ovote
0,1968.0,Alaska,2,1.0,0.0,AK,4081.0,4615.0,0.0
1,1968.0,Alaska,2,2.0,0.0,AK,601.0,1282.0,0.0
2,1968.0,Alaska,2,3.0,0.0,AK,1194.0,1248.0,0.0
3,1968.0,Alaska,2,4.0,0.0,AK,5631.0,5353.0,0.0
4,1968.0,Alaska,2,5.0,0.0,AK,673.0,696.0,0.0


# Wrangle Data

In [6]:

def getlean(df):
    '''Partisan lean according to FiveThirtyEight is average difference between how a state 
    or district votes and how the country votes overall w/ the current presidental election results weighted at 50%,
    the previous weighted at 25%, and the results from elections for the state legislature weighted at 25%
    The assumption here, is that using congressional data to predict congressional results will introduce bias,
    such as encumbancy advantage.
    
    We'll try and recreate that here, on the state level.
    
    This function will produce the 
    '''
    #calculate total lean for each year
    year_totals = df.groupby('year').sum()    
    #country_lean[year] = yeardf.candidatevotes_rep.sum()/yeardf.totalvotes.sum() - yeardf.candidatevotes_dem.sum()/yeardf.totalvotes.sum()
    year_totals['lean'] = year_totals.candidatevotes_rep / year_totals.totalvotes - year_totals.candidatevotes_dem / year_totals.totalvotes
    
    #calculate raw lean for each unit
    df['raw_lean'] = df.candidatevotes_rep/df.totalvotes - df.candidatevotes_dem/df.totalvotes
    
    #subtract the country's total lean from the lean of that unit (eg county) to get the adjusted lean for that year
    df['adjusted_lean'] = [df.loc[x,'raw_lean'] - year_totals.loc[df.loc[x,'year'],'lean'] for x in df.index]
    
    #ad compound lean
    #df['compound_lean'] = df.adjusted_lean * .75
    
    #for year in year_totals.index[1:]:
        #df.loc[df.year == year].loc[:,'compound_lean'] = df.loc[df.year == year].compound_lean + df.loc[df.year == year-1].adjusted_lean*.25 
    
    #df['compound_lean'] < start here: 
    return df

#Create house lean by district by year
#cut down to needed columns
cut_dist = house_by_district.loc[:,['year','state_po','district','party','candidatevotes','totalvotes']]

#create dem and repub data sets 
dem_dist = cut_dist.loc[cut_dist.party == 'democrat'].drop('party', axis=1)
rep_dist = cut_dist.loc[cut_dist.party == 'republican'].drop('party',axis=1)

#Merge them (there was probably a way to melt of something)
distlean = pd.merge(rep_dist, dem_dist, on=['year','state_po','district','totalvotes'], suffixes=('_rep','_dem'))


#Calculate the lean rep/total - dem/total
#distlean['lean'] = distlean.candidatevotes_rep/distlean.totalvotes - distlean.candidatevotes_dem/distlean.totalvotes
distlean = getlean(distlean)
distlean.head()

#save this as a csv for good measure

,year,state_po,district,candidatevotes_rep,totalvotes,candidatevotes_dem,raw_lean,adjusted_lean
0,1976,AL,1,98257,157170,58906,0.250372,0.347730
1,1976,AL,2,90069,156362,66288,0.152089,0.249447
2,1976,AL,4,34531,176022,141490,-0.607646,-0.510288
3,1976,AL,6,92113,162518,69384,0.139855,0.237213
4,1976,AK,0,83722,118208,34194,0.418990,0.516348


In [7]:
distlean.loc[distlean.year == 2004].head()

,year,state_po,district,candidatevotes_rep,totalvotes,candidatevotes_dem,raw_lean,adjusted_lean
5222,2004,AL,1,161067,255164,93938,0.263082,0.245368
5223,2004,AL,2,177086,247947,70562,0.429624,0.411910
5224,2004,AL,3,150411,245784,95240,0.224469,0.206755
5225,2004,AL,4,191110,255724,64278,0.495972,0.478258
5226,2004,AL,5,74145,275459,200999,-0.460519,-0.478233


In [8]:
#yep that's what that is
#Create county lean by county by year
cut_county = pres_by_county.drop(['office','state','version','candidate'], axis=1)
dem_county = cut_county.loc[cut_county.party == 'democrat'].drop('party', axis=1)
rep_county = cut_county.loc[cut_county.party == 'republican'].drop('party', axis=1)
countylean = pd.merge(rep_county, dem_county, on=['year','state_po','county','totalvotes','FIPS'], suffixes=('_rep','_dem'))
countylean.head()

#countylean['lean'] = countylean.candidatevotes_rep/countylean.totalvotes - countylean.candidatevotes_dem/countylean.totalvotes
countylean = getlean(countylean)

#save this as a csv for good measure
countylean.to_csv('Data/Output/county_pres_lean.csv')

#print to take a look at it
print(countylean.shape)
countylean.head()

(15789, 9)


,year,state_po,county,FIPS,candidatevotes_rep,totalvotes,candidatevotes_dem,raw_lean,adjusted_lean
0,2000,AL,Autauga,1001.0,11993.0,17208,4942.0,0.409751,0.414913
1,2000,AL,Baldwin,1003.0,40872.0,56480,13997.0,0.475832,0.480994
2,2000,AL,Barbour,1005.0,5096.0,10395,5188.0,-0.008850,-0.003689
3,2000,AL,Bibb,1007.0,4273.0,7101,2710.0,0.220110,0.225271
4,2000,AL,Blount,1009.0,12667.0,17973,4977.0,0.427864,0.433025


In [60]:
#Wrangle legislateive elections data
#group by year and state
state_leg_totals = state_leg.groupby(['year','state','sfips','sab']).sum().reset_index().drop(['sen','sfips','dno'], axis=1)

#match column headers with other dataframes
state_leg_totals.columns = ['year','state','state_po','candidatevotes_dem','candidatevotes_rep','voteso']

#aggrigate totals from off year elections into subsequent election years.
state_leg_totals.loc[:,'year'] = [x + (4-(x - 1968)%4) if (x-1968)%4 > 0 else x for x in state_leg_totals.year]
state_leg_totals = state_leg_totals.groupby(['year','state','state_po']).sum().reset_index()

#create totalvotes column by summing dem republican, and other candidate votes 
state_leg_totals['totalvotes'] = state_leg_totals.loc[:,'candidatevotes_dem':'voteso'].transpose().sum()

#calculate lean relative to the country
state_leg_lean = getlean(state_leg_totals)

#save as csv for good measure
state_leg_lean.to_csv('Data/Output/state_leg_lean.csv')

#state = state_leg_lean.state_po == 'CA'
#state_leg_lean.loc[state]
year = state_leg_lean.year == 2016
state_leg_lean.loc[year,:].head()
#state_leg_totals

,year,state,state_po,candidatevotes_dem,candidatevotes_rep,voteso,totalvotes,raw_lean,adjusted_lean
562,2016.0,Alabama,AL,717221.0,1303957.0,37876.0,2059054.0,0.284954,0.238812
563,2016.0,Alaska,AK,330811.0,444394.0,60309.0,835514.0,0.135944,0.089802
564,2016.0,Arizona,AZ,3868697.0,5368308.0,157055.0,9394060.0,0.159634,0.113492
565,2016.0,Arkansas,AR,389509.0,508946.0,50009.0,948464.0,0.125927,0.079785
566,2016.0,California,CA,18234768.0,11537911.0,330076.0,30102755.0,-0.222467,-0.268608


In [59]:
def compound_lean(df,leg_df):
    '''takes a df with year, state, and adjusted lean, returns a df with compound lean
    The adjusted lean of each election year wieghted at .75 with the previous weighted at .25'''
    
    #create two data frames - one with all but the firs year, and one with all but the last year 
    new_df = df.loc[df.year != df.year.unique()[0]]
    old_df = df.loc[df.year != df.year.unique()[-1]]
    
    #Shift the year column for years back to match the subsequent years.
    old_df.loc[:,'year'] = old_df.year + 4
    
    #index the frames using year and state codes
    old_df = old_df.set_index(['year','state_po']).sort_index()
    new_df = new_df.set_index(['year','state_po']).sort_index()    
    leg_df = leg_df.set_index(['year','state_po']).sort_index()
    
    #adjust years in state elections to fit presidental elections
    #leg_df.loc[:,'year'] = [x + (4-(x - 1968)%4) if (x-1968)%4 > 0 else x for x in leg_df.year]
    #print(leg_df.loc[leg_df.state_po == 'MD'])
    
    #match index of state_leg with others, also group into... oh this shouldn't be done here.
    #leg_df_group = leg_df.groupby(['year','state_po']).mean().sort_index()
    #dups = leg_df.loc[:,['year','state_po']].duplicated(keep='first')
    #leg_df_dup = leg_df[~dups].set_index(['year','state_po'])
    
    #copy new_df
    lean_df = new_df
    
    lean_df['compound_lean'] = new_df.adjusted_lean*.50 + old_df.adjusted_lean*.25 + leg_df.adjusted_lean*.25
    
    return lean_df.reset_index()


#Create presidential lean by state
state_pres_lean = getlean(countylean.groupby(['year','state_po',]).sum().reset_index().drop('FIPS', axis=1))

state_pres_leg_lean = compound_lean(state_pres_lean,state_leg_lean)
    
#Save as csv
state_pres_leg_lean.to_csv('Data/Output/state_pres_leg_lean.csv')

#print what it looks like
state_pres_leg_lean.loc[state_pres_leg_lean.year == 2016].head()


,year,state_po,candidatevotes_rep,totalvotes,candidatevotes_dem,raw_lean,adjusted_lean,compound_lean
153,2016,AK,163347.0,318056,116180.0,0.148298,0.169263,0.169381
154,2016,AL,1318250.0,2123367,729547.0,0.277250,0.298215,NaN
155,2016,AR,684872.0,1129896,380494.0,0.269386,0.290351,0.224409
156,2016,AZ,1252401.0,2628756,1161167.0,0.034706,0.055671,0.085514
157,2016,CA,4483810.0,14181595,8753788.0,-0.301093,-0.280128,-0.264436


In [ ]:
#Find a way to join the above to create a rough approximation of lean at the district level

#Grab the useful columns out of the county assignments, rename them
county_merge = county_district.loc[:,['state','FIPS','county','cd']]
county_merge.columns = ['state_po','FIPS','county','district']

#grab theuseful columns out of district lean
coldrop = ['candidatevotes_rep','totalvotes','candidatevotes_dem']
dist = distlean.drop(coldrop, axis=1)

#grab the useful columns out of county lean
#county = countylean.drop(coldrop, axis=1)

county_dist_merge = pd.merge(countylean, county_merge, on='FIPS', how='left')


#county_dist_merge = pd.merge(county, dist, on=['state_po','district'])

#Sanity Checks
print('sanity checks')
print(countylean.shape, county_dist_merge.shape, county_merge.shape)
error = county_dist_merge.county_x != county_dist_merge.county_y
print(county_dist_merge.county_x[0], county_dist_merge.county_y[0])
print(county_dist_merge.loc[error].head())


county_dist_merge.head()

In [9]:
#Find split counties
#find repeat values
dups = county_dist_merge[['year','FIPS']].duplicated(keep=False)
print('duplicated counties:', dups.sum())

#get a list of districts with repeated counties
dists_w_dups = county_dist_merge.loc[dups,['state_po_x','district']].drop_duplicates()
print('districts with duplicate counties:',len(dists_w_dups))


duplicated counties: 5170
districts with duplicate counties: 421


Apparently nearly every district shares a county with another district. We can't drop every district with a split party. We can try grouping only the counties that aren't split by district, or try using the open elections platform.

Perhaps compare the lean results from 2018 to the actual lean, and see what the difference is w double counting. 

In [10]:
#ignore sanity check for now.... let's get some presidential vote by district.
#Note... you are double counting split counties with this method.
pres_by_district = county_dist_merge.drop('FIPS', axis=1).groupby(['year','state_po_x','district']).sum()
pres_by_district = getlean(pres_by_district)
pres_by_district

candidatevotes_rep  totalvotes  candidatevotes_dem  \
year state_po_x district                                                       
2000 AL         1.0                 141267.0      235184             88966.0   
                2.0                 152087.0      258733            102757.0   
                3.0                 153493.0      285452            126635.0   
                4.0                 187623.0      327555            133044.0   
                5.0                 128082.0      229676             96192.0   
                6.0                 215530.0      380624            157669.0   
                7.0                 247759.0      504314            247803.0   
     AR         1.0                 119005.0      259883            134429.0   
                2.0                 112852.0      229133            109658.0   
                3.0                 131266.0      218536             79969.0   
                4.0                 158008.0      307199            140705.0   
     AZ         1.0                 741877.0     1458844            652924.0   
                2.0                 142759.0      320881            161048.0   
                3.0                 643720.0     1268174            571309.0   
                4.0                 592028.0     1106229            469390.0   
                5.0                 479967.0      901699            386683.0   
                6.0                 479967.0      901699            386683.0   
                7.0                 479967.0      901699            386683.0   
                8.0                 479967.0      901699            386683.0   
                9.0                 479967.0      901699            386683.0   
     CA         1.0                 233522.0      391868            134837.0   
                2.0                 141758.0      424634            242964.0   
                3.0                 315591.0      702632            350577.0   
                4.0                 313048.0      558371            218487.0   
                5.0                 285838.0      782862            455563.0   
                6.0                 218676.0      492986            246539.0   
                7.0                 195619.0      431550            212792.0   
                8.0                 228766.0      467079            219189.0   
                9.0                 418765.0      980267            516906.0   
                10.0                148961.0      295506            136224.0   
...                                      ...         ...                 ...   
2016 VA         3.0                 163395.0      445814            258757.0   
                4.0                 273068.0      689541            378801.0   
                5.0                 225456.0      411701            166508.0   
                6.0                 231361.0      384092            130094.0   
                7.0                 242612.0      504517            233050.0   
                8.0                 192505.0      756066            510210.0   
                9.0                 244683.0      363500            102386.0   
                10.0                342600.0     1010781            600846.0   
                11.0                233133.0      759751            475644.0   
     WA         1.0                 409929.0     1506313            990579.0   
                2.0                 214743.0      561464            300389.0   
                3.0                 202663.0      438893            200995.0   
                4.0                 156600.0      272540             96556.0   
                5.0                 168422.0      321927            125589.0   
                6.0                 248417.0      588975            290040.0   
                7.0                 344594.0     1340672            903549.0   
                8.0                 400980.0     1414862            916299.0   
                9.0       

In [11]:
#Remove double counts?

#combine with other, stuff